# The entire Code is taken and adapted from the following sources :
1. "https://github.com/yao8839836/kg-bert"
2. "https://huggingface.co/docs/transformers/v4.25.1/en/index"
3. "https://huggingface.co/docs/datasets/index"
4. "https://huggingface.co/docs/tokenizers/index"



# **Setup**

In [1]:
print('Mounting google drive...')
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive"
!mkdir KG-Models
%cd "/content/drive/MyDrive/KG-Models"

Mounting google drive...
Mounted at /content/drive
/content/drive/MyDrive
mkdir: cannot create directory ‘KG-Models’: File exists
/content/drive/MyDrive/KG-Models


In [3]:
import sys
if 'google.colab' in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

In [5]:
!pip install ipython-autotime

%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 34.8 MB/s 
time: 438 µs (started: 2022-12-19 12:41:42 +00:00)


In [4]:
# progress bars in model download and training scripts
!pip install tqdm~=4.64.0
# Accessing files from S3 directly.
!pip install boto3
# Used for downloading models over HTTP
!pip install requests
# bert

# wikipedia
!pip install wikipedia~=1.4.0
!pip install transformers~=4.21.1
!pip install tqdm
!pip install ujson
!pip install seqeval
!pip install sentencepiece
!pip install datasets
!pip install evaluate
!pip install wandb


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 132 kB 29.1 MB/s 
     |████████████████████████████████| 79 kB 10.2 MB/s 
     |████████████████████████████████| 10.3 MB 73.2 MB/s 
     |████████████████████████████████| 140 kB 91.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.13 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 127 kB 25.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.13
    Uninstalling urllib3-1.26.13:
      Successfully uninstalled urllib3-1.26.13


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=e1089a3d163916b0d5631cca659f4afd574fea1d181712d5319a2707f3323129
  Stored in directory: /root/.cache/pip/wheels/07/93/05/72c05349177dca2e0ba31a33ba4f7907606f7ddef303517c6a
Successfully built wikipedia
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 29.7 MB/s 
     |████████████████████████████████| 6.6 MB 80.7 MB/s 
     |████████████████████████████████| 182 kB 91.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 52 kB 1.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheel

In [6]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!
time: 2.11 ms (started: 2022-12-19 12:41:42 +00:00)


In [7]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Dec 19 12:41:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    29W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
!pip install wandb
import wandb
wandb.login()
#1c936ec91786bdd6cc3afad6d11df354cf92fbdc
%env WANDB_LOG_MODEL=true

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


env: WANDB_LOG_MODEL=true
time: 21 s (started: 2022-12-19 12:41:43 +00:00)


In [9]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

time: 3.95 s (started: 2022-12-19 12:42:04 +00:00)


In [10]:
torch.cuda.is_available()

True

time: 2.64 ms (started: 2022-12-19 12:42:08 +00:00)


In [11]:
from __future__ import absolute_import, division, print_function

import argparse
import csv
import logging
import os
import random
import sys

import numpy as np
import evaluate
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange

from torch.nn import CrossEntropyLoss, MSELoss
from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import matthews_corrcoef, f1_score
from sklearn import metrics

from transformers import AdamW, WEIGHTS_NAME, CONFIG_NAME
from transformers import DebertaTokenizer, DebertaForSequenceClassification
from transformers import XLNetTokenizer, XLNetForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import get_linear_schedule_with_warmup
#from pytorch_transformers.optimization import WarmupLinearSchedule

from datasets import Dataset, DatasetDict
from datasets import load_metric

os.environ['CUDA_VISIBLE_DEVICES'] = '6'
#torch.backends.cudnn.deterministic = True

logger = logging.getLogger(__name__)


time: 4.57 s (started: 2022-12-19 12:42:08 +00:00)


In [12]:
# The entire Code is taken and adapted from the following source : "https://github.com/yao8839836/kg-bert"
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text, label=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            text_c: (Optional) string. The untokenized text of the third sequence.
            Only must be specified for sequence triple tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        #Changing Here
        self.text = text
        self.label = label



class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_labels(self, data_dir):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []
            for line in reader:
                if sys.version_info[0] == 2:
                    line = list(unicode(cell, 'utf-8') for cell in line)
                lines.append(line)
            return lines
    


class KGProcessor(DataProcessor):
    """Processor for knowledge graph data set."""

    def __init__(self):
        self.labels = set()

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "train.tsv")), "train", data_dir)

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev", data_dir)

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "test.tsv")), "test", data_dir)

    def get_relations(self, data_dir):
        """Gets all labels (relations) in the knowledge graph."""
        # return list(self.labels)
        with open(os.path.join(data_dir, "relations.txt"), 'r') as f:
            lines = f.readlines()
            relations = []
            for line in lines:
                relations.append(line.strip())
        return relations

    def get_entities(self, data_dir):
        """Gets all entities in the knowledge graph."""
        # return list(self.labels)
        with open(os.path.join(data_dir, "entities.txt"), 'r') as f:
            lines = f.readlines()
            entities = []
            for line in lines:
                entities.append(line.strip())
        return entities

    

    def get_train_triples(self, data_dir):
        """Gets training triples."""
        return self._read_tsv(os.path.join(data_dir, "train.tsv"))

    def get_dev_triples(self, data_dir):
        """Gets validation triples."""
        return self._read_tsv(os.path.join(data_dir, "dev.tsv"))

    def get_test_triples(self, data_dir):
        """Gets test triples."""
        return self._read_tsv(os.path.join(data_dir, "test.tsv"))
    
    def _create_examples(self, lines, set_type, data_dir):
        """Creates examples for the training and dev sets."""
        # entity to text
        ent2text = {}
        with open(os.path.join(data_dir, "entity2text.txt"), 'r') as f:
            ent_lines = f.readlines()
            for line in ent_lines:
                temp = line.strip().split('\t')
                ent2text[temp[0]] = temp[1]

        if data_dir.find("FB15") != -1:
            with open(os.path.join(data_dir, "entity2text.txt"), 'r') as f:
                ent_lines = f.readlines()
                for line in ent_lines:
                    temp = line.strip().split('\t')
                    #first_sent_end_position = temp[1].find(".")
                    ent2text[temp[0]] = temp[1]#[:first_sent_end_position + 1]              

        examples = []
        for (i, line) in enumerate(lines):
            guid = "%s-%s" % (set_type, i)
            text_a = ent2text[line[0]]
            text_b = ent2text[line[2]]
            label = line[1]
            text="[CLS] "+text_a+" [SEP] "+text_b+" [SEP]"
            self.labels.add(label)
            examples.append(
                InputExample(guid=guid, text=text, label=label))
        return examples




time: 6.13 ms (started: 2022-12-19 12:42:12 +00:00)


In [13]:
data_dir = "/content/drive/MyDrive/kg-bert/data_sample_relation/FB15k-237"
processors = {
        "kg": KGProcessor,
    }
processor = processors['kg']()

time: 783 µs (started: 2022-12-19 12:42:13 +00:00)


In [14]:
train_examples=processor.get_train_examples(data_dir)
eval_examples=processor.get_dev_examples(data_dir)
test_examples=processor.get_test_examples(data_dir)

time: 4.62 s (started: 2022-12-19 12:42:16 +00:00)


In [15]:
label_list = processor.get_relations(data_dir)
num_labels = len(label_list)
label_to_id = {v: i for i, v in enumerate(label_list)}

time: 570 ms (started: 2022-12-19 12:42:21 +00:00)


In [16]:
object_dict_train={'guid':[id.guid for id in train_examples],'triple_texts':[x.text for x in train_examples],'labels':[label_to_id[l.label] for l in train_examples]}
object_dict_eval={'guid':[id.guid for id in eval_examples],'triple_texts':[x.text for x in eval_examples],'labels':[label_to_id[l.label] for l in eval_examples]}
object_dict_test={'guid':[id.guid for id in test_examples],'triple_texts':[x.text for x in test_examples],'labels':[label_to_id[l.label] for l in test_examples]}

time: 22.6 ms (started: 2022-12-19 12:42:21 +00:00)


In [17]:
dataset_train = Dataset.from_dict(object_dict_train)
dataset_eval = Dataset.from_dict(object_dict_eval)
dataset_test = Dataset.from_dict(object_dict_test)
print(dataset_train[0])
print(dataset_eval[0])
print(dataset_test[0])

{'guid': 'train-0', 'triple_texts': '[CLS] Dominican Republic [SEP] Republic [SEP]', 'labels': 26}
{'guid': 'dev-0', 'triple_texts': '[CLS] American Pie [SEP] Romance Film [SEP]', 'labels': 140}
{'guid': 'test-0', 'triple_texts': '[CLS] Zürich [SEP] October [SEP]', 'labels': 52}
time: 50.4 ms (started: 2022-12-19 12:42:21 +00:00)


In [18]:
dataset = DatasetDict({"train":dataset_train,"eval":dataset_eval,"test":dataset_test})
dataset

DatasetDict({
    train: Dataset({
        features: ['guid', 'triple_texts', 'labels'],
        num_rows: 50000
    })
    eval: Dataset({
        features: ['guid', 'triple_texts', 'labels'],
        num_rows: 17535
    })
    test: Dataset({
        features: ['guid', 'triple_texts', 'labels'],
        num_rows: 20466
    })
})

time: 2.68 ms (started: 2022-12-19 12:42:25 +00:00)


# **Deberta Implementation**

In [ ]:
tokenizer = DebertaTokenizer.from_pretrained("microsoft/deberta-base")

In [ ]:
encoded_dataset = dataset.map(lambda x: tokenizer(x['triple_texts'], truncation=True, padding='max_length', max_length=50,return_tensors='pt'), remove_columns=['guid', 'triple_texts'],batched=True)
encoded_dataset

  0%|          | 0/50 [00:00<?, ?ba/s]

  0%|          | 0/18 [00:00<?, ?ba/s]

  0%|          | 0/21 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
    eval: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 17535
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 20466
    })
})

In [ ]:
model = DebertaForSequenceClassification.from_pretrained('microsoft/deberta-base', num_labels=num_labels).to(device)

Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.bias']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.weight', 'pooler.d

In [ ]:


args = TrainingArguments(
    output_dir='./output_FB15K_deberta_relationprediction_final/',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=64,  # batch size per device during training
    per_device_eval_batch_size=512,   # batch size for evaluation
    warmup_steps=0.1,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs', 
    report_to = 'wandb',                     # enable logging to W&B
    overwrite_output_dir = True,
    learning_rate = 5e-5,  
    save_strategy="no",
     # we can customize learning rate
    evaluation_strategy = 'epoch',  
    logging_strategy = 'epoch',             
    load_best_model_at_end = False,
    metric_for_best_model = 'accuracy',
    greater_is_better = True,
    run_name = 'deberta_relation_prediction'            # name of the W&B run
)

In [ ]:


'''metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    metric = evaluate.load("accuracy")
    return metric.compute(predictions=predictions, references=labels)'''



def compute_metrics(eval_pred):
  metric1 = evaluate.load("precision")
  metric2 = evaluate.load("recall")
  metric3 = evaluate.load("f1")
  metric4 = evaluate.load("accuracy")

  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  precision = metric1.compute(predictions=predictions, references=labels,average="weighted")["precision"]
  recall = metric2.compute(predictions=predictions, references=labels, average="weighted")["recall"]
  f1 = metric3.compute(predictions=predictions, references=labels, average="weighted")["f1"]
  accuracy = metric4.compute(predictions=predictions, references=labels)["accuracy"]

  return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}

In [ ]:
num_labels

237

In [ ]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, batch,return_outputs=False):
        num_labels=237
        #batch = tuple(t.to(device) for t in batch)
        # forward pass
        outputs = model(**batch)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, num_labels), batch['labels'].view(-1))
        return (loss, outputs) if return_outputs else loss

In [ ]:
from transformers.data.data_collator import default_data_collator
trainer = CustomTrainer(model = model,                  # model to be trained
    args = args,                    # training args
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["eval"],
    tokenizer=tokenizer,            # for padding batched data
    data_collator=default_data_collator,
    compute_metrics=compute_metrics)

In [ ]:
wandb.finish()

eval/accuracy,▁▄▇▇██
eval/f1,▁▄▆▇▇█
eval/loss,█▄▂▁▁▁
eval/precision,▁▅▆▇▇█
eval/recall,▁▄▇▇██
eval/runtime,█▂▁▁▁▁
eval/samples_per_second,▁▇████
eval/steps_per_second,▁▇████
train/epoch,▁▁▂▂▄▄▅▅▇▇███
train/global_step,▁▁▂▂▄▄▅▅▇▇███
train/learning_rate,█▇▅▄▂▁


In [ ]:
wandb.init()

In [ ]:
trainer.train()
#trainer.save_model(f'./output_FB15K_deberta_relationprediction')

***** Running training *****
  Num examples = 50000
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 2346


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.902600,0.384455,0.875933,0.894611,0.877198,0.894611
2,0.557600,0.276389,0.911610,0.917993,0.909898,0.917993
3,0.461100,0.263077,0.916551,0.923695,0.916817,0.923695


***** Running Evaluation *****
  Num examples = 17535
  Batch size = 512
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Evaluation *****
  Num examples = 17535
  Batch size = 512
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

TrainOutput(global_step=2346, training_loss=0.6404234747142743, metrics={'train_runtime': 487.6766, 'train_samples_per_second': 307.581, 'train_steps_per_second': 4.811, 'total_flos': 4898606527425600.0, 'train_loss': 0.6404234747142743, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 17535
  Batch size = 512


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.3357827067375183,
 'eval_precision': 0.8937808083352444,
 'eval_recall': 0.9051040775591673,
 'eval_f1': 0.8938627905183637,
 'eval_accuracy': 0.9051040775591673,
 'eval_runtime': 16.8892,
 'eval_samples_per_second': 1038.237,
 'eval_steps_per_second': 2.072,
 'epoch': 3.0}

In [ ]:
trainer.predict(encoded_dataset['test'])

***** Running Prediction *****
  Num examples = 20466
  Batch size = 512


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


PredictionOutput(predictions=array([[-2.8523474 , -2.3233287 , -3.6852994 , ..., -1.507836  ,
        -4.705618  , -4.109329  ],
       [-2.9349678 , -1.5889304 , -3.2456505 , ...,  1.4016768 ,
        -5.534763  , -5.7039943 ],
       [-2.1163046 ,  1.1561831 , -3.0887268 , ..., -2.3090563 ,
        -4.346329  , -3.9936297 ],
       ...,
       [-2.206653  , -0.69181305, -2.3240337 , ...,  0.56163347,
        -4.346863  , -3.3549979 ],
       [-4.362925  , -2.7390203 , -2.5959568 , ..., -4.615742  ,
        -1.5265325 , -5.4592395 ],
       [-2.272438  , -1.9587926 , -3.0435786 , ..., -1.1567065 ,
        -4.4472737 , -3.9040473 ]], dtype=float32), label_ids=array([ 52, 193, 190, ...,  49, 125, 198]), metrics={'test_loss': 0.33797383308410645, 'test_precision': 0.8921381533349191, 'test_recall': 0.9034984852926805, 'test_f1': 0.891875941043077, 'test_accuracy': 0.9034984852926805, 'test_runtime': 19.2857, 'test_samples_per_second': 1061.2, 'test_steps_per_second': 2.074})

In [ ]:
wandb.finish()

eval/accuracy,▁▇██
eval/f1,▁▆██
eval/loss,█▂▁▁
eval/precision,▁▆██
eval/recall,▁▇██
eval/runtime,█▁▁▁
eval/samples_per_second,▁███
eval/steps_per_second,▁███
train/epoch,▁▁▅▅████
train/global_step,▁▁▅▅████
train/learning_rate,█▅▁


# Hyperparameter Optimisation Deberta

In [ ]:
wandb.init()

In [ ]:
def wandb_hp_space(trial):
    return {
        "method": "random",
        "metric": {"name": "objective", "goal": "minimize"},
        "parameters": {
            "learning_rate": {"values": [ 5e-5, 5e-3, 2e-5]},
            "per_device_train_batch_size": {"values": [64, 128]},
            "num_train_epochs": {"values": [6]},
        },
    }

In [ ]:

def model_init(trial):
    return DebertaForSequenceClassification.from_pretrained('microsoft/deberta-base', num_labels=num_labels)

In [ ]:
from transformers.data.data_collator import default_data_collator
trainer = CustomTrainer(model_init = model_init,                  # model to be trained
    args = args,                    # training args
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["eval"],
    tokenizer=tokenizer,            # for padding batched data
    data_collator=default_data_collator,
    compute_metrics=compute_metrics)

loading configuration file https://huggingface.co/microsoft/deberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/e313266bff73867debdfa78c78a9a4966d5e78281ac4ed7048c178b16a37eba7.fb501413b9cef9cef6babdc543bb4153cbec58d52bce077647efba3e3f14ccf3
Model config DebertaConfig {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25": "LABEL_25",
    "26"

storing https://huggingface.co/microsoft/deberta-base/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/dde0725208c11536042f6f416c538792d44a2d57d1ae399bbd1bc5867e02c465.0a3ec262cb3d4f634c72ce55f2766bb88771e6499b2512830e2e63bf19dbf97a
creating metadata file for /root/.cache/huggingface/transformers/dde0725208c11536042f6f416c538792d44a2d57d1ae399bbd1bc5867e02c465.0a3ec262cb3d4f634c72ce55f2766bb88771e6499b2512830e2e63bf19dbf97a
loading weights file https://huggingface.co/microsoft/deberta-base/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/dde0725208c11536042f6f416c538792d44a2d57d1ae399bbd1bc5867e02c465.0a3ec262cb3d4f634c72ce55f2766bb88771e6499b2512830e2e63bf19dbf97a
Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_he

In [ ]:
best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="wandb",
    hp_space=wandb_hp_space,
    n_trials=3,
    
)

wandb sweep id - gpqu2k04


Create sweep with ID: gpqu2k04
Sweep URL: https://wandb.ai/anvesha/uncategorized/sweeps/gpqu2k04


wandb: Agent Starting Run: fy192ops with config:
wandb: 	learning_rate: 5e-05
wandb: 	num_train_epochs: 6
wandb: 	per_device_train_batch_size: 128
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: anvesha. Use `wandb login --relogin` to force relogin


Trying to set _wandb in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set assignments in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set metric in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
W&B Sweep parameters: {'learning_rate': 5e-05, 'num_train_epochs': 6, 'per_device_train_batch_size': 128, '_wandb': {}, 'assignments': {}, 'metric': 'eval/loss'}
loading configuration file https://huggingface.co/microsoft/deberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/e313266bff73867debdfa78c78a9a4966d5e78281ac4ed7048c178b16a37eba7.fb501413b9cef9cef6babdc543bb4153cbec58d52bce077647efba3e3f14ccf3
Model config DebertaConfig {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
   

Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

wandb: WARNING Config item 'per_device_train_batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'num_train_epochs' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,4.629500,4.615318,0.000115,0.010721,0.000227,0.010721
2,4.579100,4.615752,0.004904,0.070031,0.009167,0.070031
3,4.565200,4.591627,0.000115,0.010721,0.000227,0.010721
4,4.557600,4.599228,0.000115,0.010721,0.000227,0.010721
5,4.552000,4.591029,0.000115,0.010721,0.000227,0.010721
6,4.546000,4.585498,0.000115,0.010721,0.000227,0.010721


***** Running Evaluation *****
  Num examples = 17535
  Batch size = 512
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Evaluation *****
  Num examples = 17535
  Batch size = 512
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Evaluation *****
  Num examples = 17535
  Batch size = 512
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples

eval/accuracy,▁█▁▁▁▁
eval/f1,▁█▁▁▁▁
eval/loss,██▂▄▂▁
eval/precision,▁█▁▁▁▁
eval/recall,▁█▁▁▁▁
eval/runtime,▄▁▇▆█▅
eval/samples_per_second,▅█▂▃▁▄
eval/steps_per_second,▄█▂▃▁▃
train/epoch,▁▁▂▂▄▄▅▅▇▇███
train/global_step,▁▁▂▂▄▄▅▅▇▇███
train/learning_rate,█▇▅▄▂▁


Run pgdnnw5p errored: ValueError('w is not supported, only azure_ml, comet_ml, mlflow, neptune, tensorboard, wandb, codecarbon are supported.')
wandb: ERROR Run pgdnnw5p errored: ValueError('w is not supported, only azure_ml, comet_ml, mlflow, neptune, tensorboard, wandb, codecarbon are supported.')


In [ ]:
best_trial

BestRun(run_id=None, objective=None, hyperparameters=None)

In [ ]:
wandb.finish()

eval/accuracy,▁▇█
eval/f1,▁▇█
eval/loss,█▂▁
eval/precision,▁▇█
eval/recall,▁▇█
eval/runtime,▁█▇
eval/samples_per_second,█▁▂
eval/steps_per_second,█▁▁
train/epoch,▁▁▅▅███
train/global_step,▁▁▅▅███
train/learning_rate,█▄▁


In [ ]:
wandb.login()

True

In [ ]:

args = TrainingArguments(
    output_dir='./output_FB15K_deberta_relationprediction_final/',          # output directory
    num_train_epochs=6,              # total # of training epochs
    per_device_train_batch_size=64,  # batch size per device during training
    per_device_eval_batch_size=512,   # batch size for evaluation
    warmup_steps=0.1,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs', 
    report_to = 'wandb',                     # enable logging to W&B
    overwrite_output_dir = True,
    learning_rate = 5e-5,  
    save_strategy="no",
     # we can customize learning rate
    evaluation_strategy = 'epoch',  
    logging_strategy = 'epoch',             
    load_best_model_at_end = False,
    metric_for_best_model = 'accuracy',
    greater_is_better = True,
    run_name = 'deberta_relation_prediction'            # name of the W&B run
)

PyTorch: setting up devices


In [ ]:
from transformers.data.data_collator import default_data_collator
trainer = CustomTrainer(model = model,                  # model to be trained
    args = args,                    # training args
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["eval"],
    tokenizer=tokenizer,            # for padding batched data
    data_collator=default_data_collator,
    compute_metrics=compute_metrics)

In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 50000
  Num Epochs = 6
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 4692
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.515100,0.292465,0.910816,0.906872,0.902812,0.906872
2,0.442100,0.249441,0.917256,0.917821,0.911936,0.917821
3,0.380700,0.233986,0.927465,0.928258,0.923825,0.928258
4,0.320600,0.244052,0.929301,0.928087,0.926043,0.928087
5,0.269000,0.252116,0.931918,0.928429,0.927423,0.928429
6,0.228200,0.261681,0.933971,0.928771,0.929010,0.928771


***** Running Evaluation *****
  Num examples = 17535
  Batch size = 512
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Evaluation *****
  Num examples = 17535
  Batch size = 512
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

TrainOutput(global_step=4692, training_loss=0.3592667282898001, metrics={'train_runtime': 981.3813, 'train_samples_per_second': 305.692, 'train_steps_per_second': 4.781, 'total_flos': 8998615890000000.0, 'train_loss': 0.3592667282898001, 'epoch': 6.0})

In [ ]:
wandb.finish()

eval/accuracy,▁▆▇███
eval/f1,▁▅▆███
eval/loss,█▃▂▁▁▁
eval/precision,▁▅▇▇██
eval/recall,▁▆▇███
eval/runtime,▁▇▅█▇▅
eval/samples_per_second,█▂▄▁▂▄
eval/steps_per_second,█▂▄▁▁▃
train/epoch,▁▁▂▂▄▄▅▅▇▇███
train/global_step,▁▁▂▂▄▄▅▅▇▇███
train/learning_rate,█▇▅▄▂▁


# best run

In [ ]:

args = TrainingArguments(
    output_dir='./output_FB15K_deberta_relationprediction/',          # output directory
    num_train_epochs=6,              # total # of training epochs
    per_device_train_batch_size=128,  # batch size per device during training
    per_device_eval_batch_size=512,   # batch size for evaluation
    warmup_steps=0.1,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs', 
    report_to = 'wandb',                     # enable logging to W&B
    overwrite_output_dir = True,
    learning_rate = 5e-5,  
    save_strategy="no",
     # we can customize learning rate
    evaluation_strategy = 'epoch',  
    logging_strategy = 'epoch',             
    load_best_model_at_end = False,
    metric_for_best_model = 'accuracy',
    greater_is_better = True,
    run_name = 'deberta_relation_prediction_best_run'            # name of the W&B run
)

In [ ]:
from transformers.data.data_collator import default_data_collator
trainer = CustomTrainer(model = model,                  # model to be trained
    args = args,                    # training args
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["eval"],
    tokenizer=tokenizer,            # for padding batched data
    data_collator=default_data_collator,
    compute_metrics=compute_metrics)

In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 50000
  Num Epochs = 6
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 2346
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


wandb: Currently logged in as: anvesha. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,1.921100,0.550689,0.832879,0.868606,0.842139,0.868606
2,0.673600,0.317235,0.893980,0.910294,0.896932,0.910294
3,0.521800,0.267958,0.908820,0.917422,0.907829,0.917422
4,0.453800,0.249279,0.919358,0.923810,0.916740,0.923810
5,0.402900,0.238637,0.924207,0.924779,0.920209,0.924779
6,0.363000,0.233029,0.929096,0.927687,0.924961,0.927687


***** Running Evaluation *****
  Num examples = 17535
  Batch size = 512


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Evaluation *****
  Num examples = 17535
  Batch size = 512
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sk

TrainOutput(global_step=2346, training_loss=0.7226869366866142, metrics={'train_runtime': 3084.5488, 'train_samples_per_second': 97.259, 'train_steps_per_second': 0.761, 'total_flos': 8998615890000000.0, 'train_loss': 0.7226869366866142, 'epoch': 6.0})

In [ ]:
trainer.save_model(f'./output_FB15K_deberta_relationprediction')

Saving model checkpoint to ./output_FB15K_deberta_relationprediction
Configuration saved in ./output_FB15K_deberta_relationprediction/config.json
Model weights saved in ./output_FB15K_deberta_relationprediction/pytorch_model.bin
tokenizer config file saved in ./output_FB15K_deberta_relationprediction/tokenizer_config.json
Special tokens file saved in ./output_FB15K_deberta_relationprediction/special_tokens_map.json


In [ ]:
trainer.predict(encoded_dataset['test'])

***** Running Prediction *****
  Num examples = 20466
  Batch size = 512


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


PredictionOutput(predictions=array([[-2.66427   ,  0.70199144, -3.6944854 , ..., -4.3574667 ,
        -2.7998753 , -6.1139054 ],
       [-3.7776713 , -0.847145  , -3.3968697 , ...,  0.5599447 ,
        -4.9396653 , -4.4481425 ],
       [-1.9706094 ,  0.53271073, -4.337755  , ..., -0.91045606,
        -4.303894  , -5.3691235 ],
       ...,
       [-4.3198986 , -0.7664739 , -4.542214  , ..., -0.7686298 ,
        -2.656749  , -1.9934227 ],
       [-2.694811  , -3.037229  , -3.464739  , ..., -5.650209  ,
        -1.98979   , -4.080781  ],
       [-1.1739272 , -0.49074626, -2.458425  , ..., -1.2082049 ,
        -4.010508  , -5.3898134 ]], dtype=float32), label_ids=array([ 52, 193, 190, ...,  49, 125, 198]), metrics={'test_loss': 0.2346828132867813, 'test_precision': 0.9280548051215988, 'test_recall': 0.9280758330890257, 'test_f1': 0.9252858609534143, 'test_accuracy': 0.9280758330890257, 'test_runtime': 42.2631, 'test_samples_per_second': 484.252, 'test_steps_per_second': 0.946})

In [90]:
tokenizer = DebertaTokenizer.from_pretrained("./output_FB15K_deberta_relationprediction")
model = DebertaForSequenceClassification.from_pretrained('./output_FB15K_deberta_relationprediction', num_labels=num_labels)

time: 1.5 s (started: 2022-12-19 15:34:51 +00:00)


In [91]:
encoded_dataset = dataset.map(lambda x: tokenizer(x['triple_texts'], truncation=True, padding='max_length', max_length=30,return_tensors='pt'), remove_columns=['guid', 'triple_texts'],batched=True)
encoded_dataset

  0%|          | 0/50 [00:00<?, ?ba/s]

  0%|          | 0/18 [00:00<?, ?ba/s]

  0%|          | 0/21 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
    eval: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 17535
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 20466
    })
})

time: 19.2 s (started: 2022-12-19 15:34:55 +00:00)


In [110]:
#Relation Prediction using test data 
from transformers.data.data_collator import default_data_collator

test_dataloader = DataLoader(encoded_dataset["test"], collate_fn=default_data_collator, batch_size=128)
all_input_ids = torch.tensor([f for f in encoded_dataset['test']['input_ids']], dtype=torch.long)
all_attention_mask = torch.tensor([f for f in encoded_dataset['test']['attention_mask']], dtype=torch.long)
all_token_type_ids = torch.tensor([f for f in encoded_dataset['test']['token_type_ids']], dtype=torch.long)
all_label_ids = torch.tensor([f for f in encoded_dataset['test']['labels']], dtype=torch.long)

time: 902 ms (started: 2022-12-19 17:02:55 +00:00)


In [93]:
eval_data = TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids, all_label_ids)
        # Run prediction for full data
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=128)

time: 735 µs (started: 2022-12-19 15:35:22 +00:00)


In [94]:
label_list = processor.get_relations(data_dir)
num_labels = len(label_list)

time: 2.4 ms (started: 2022-12-19 15:35:24 +00:00)


In [54]:
label_map = {label : i for i, label in enumerate(label_list)}

time: 539 µs (started: 2022-12-19 14:09:43 +00:00)


In [64]:
labels=np.array(labels.cpu())
labels

array([ 52, 193, 190, 231, 231,   4,  89, 154, 140, 227, 190, 150,  44,
        97, 184,  29, 185,  46, 136, 136, 221,  55, 125, 209, 209,  19,
       211, 190,  28,  64,  30, 221, 132,  89,  89,   9,  79,  79,  94,
       230, 190, 194, 163, 213, 213, 124,   9,  47,   8,  79,  89, 229,
        81,  54,   4, 225, 107,  71, 127, 231,   9,  77,  54, 112, 209,
        89, 104, 109,   4,   9,  89, 137, 210, 190,   3,   3,   4, 100,
       167, 170,   4, 175,   9,  54,   4,   9, 131, 139, 143, 116, 211,
        32,  85, 162,  54,  54,  97,  55,  18,  89, 104, 211, 190, 182,
       227,  48, 143,  83, 232,  54, 142,  27, 205, 100,  71,  30, 143,
        70, 180, 198, 143,  47,  62,  33, 189,   4,  54,  18])

time: 3.89 ms (started: 2022-12-19 14:32:12 +00:00)


In [111]:
# The entire Code is taken and adapted from the following source : "https://github.com/yao8839836/kg-bert"
train_triples = processor.get_train_triples(data_dir)
dev_triples = processor.get_dev_triples(data_dir)
test_triples = processor.get_test_triples(data_dir)
all_triples = train_triples + dev_triples + test_triples

all_triples_str_set = set()
for triple in all_triples:
            triple_str = '\t'.join(triple)
            all_triples_str_set.add(triple_str)
model.to(device)
model.eval()
eval_loss = 0
nb_eval_steps = 0
preds = []

for input_ids, attention_mask, token_type_ids, labels in tqdm(eval_dataloader, desc="Testing"):
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            labels = labels.to(device)

            with torch.no_grad():
                output = model(input_ids, token_type_ids, attention_mask, labels=None)
            logits = output.get("logits")
            loss_fct = CrossEntropyLoss()
            tmp_eval_loss = loss_fct(logits.view(-1, num_labels), labels.view(-1))
            
            eval_loss += tmp_eval_loss.mean().item()
            nb_eval_steps += 1
            if len(preds) == 0:
                preds.append(logits.detach().cpu().numpy())
            else:
                preds[0] = np.append(
                    preds[0], logits.detach().cpu().numpy(), axis=0)

eval_loss = eval_loss / nb_eval_steps
preds = preds[0]
print(preds, preds.shape)
        
all_label_ids = all_label_ids.numpy()
ranks = []
filter_ranks = []
hits = []
hits_filter = []
for i in range(10):
            hits.append([])
            hits_filter.append([])

for i, pred in enumerate(preds):
            rel_values = torch.tensor(pred)
            _, argsort1 = torch.sort(rel_values, descending=True)
            argsort1 = argsort1.cpu().numpy()
            
            rank = np.where(argsort1 == all_label_ids[i])[0][0]
            #print(argsort1, all_label_ids[i], rank)
            ranks.append(rank + 1)
            test_triple = test_triples[i]
            filter_rank = rank
            for tmp_label_id in argsort1[:rank]:
                tmp_label = label_list[tmp_label_id]
                tmp_triple = [test_triple[0], tmp_label, test_triple[2]]
                #print(tmp_triple)
                tmp_triple_str = '\t'.join(tmp_triple)
                if tmp_triple_str in all_triples_str_set:
                    filter_rank -= 1
            filter_ranks.append(filter_rank + 1)

            for hits_level in range(10):
                if rank <= hits_level:
                    hits[hits_level].append(1.0)
                else:
                    hits[hits_level].append(0.0)

                if filter_rank <= hits_level:
                    hits_filter[hits_level].append(1.0)
                else:
                    hits_filter[hits_level].append(0.0)   

print("Raw mean rank: ", np.mean(ranks)*0.01)
print("Filtered mean rank: ", np.mean(filter_ranks)*0.01)
for i in [0,9]:
            print('Raw Hits @{0}: {1}'.format(i+1, np.mean(hits[i])*1000))
            print('hits_filter Hits @{0}: {1}'.format(i+1, np.mean(hits_filter[i])*1000))


Testing: 100%|██████████| 160/160 [00:37<00:00,  4.25it/s]


[[-1.5013026  -0.02726077 -2.1695752  ... -2.186095   -2.8690476
  -2.933162  ]
 [-1.5013026  -0.02726077 -2.1695752  ... -2.186095   -2.8690476
  -2.933162  ]
 [-1.5013026  -0.02726077 -2.1695752  ... -2.186095   -2.8690476
  -2.933162  ]
 ...
 [-1.5013027  -0.02726076 -2.1695755  ... -2.1860948  -2.8690476
  -2.933162  ]
 [-1.5013027  -0.02726076 -2.1695755  ... -2.1860948  -2.8690476
  -2.933162  ]
 [-1.5013027  -0.02726076 -2.1695755  ... -2.1860948  -2.8690476
  -2.933162  ]] (20466, 237)
Raw mean rank:  0.7705579986318773
Filtered mean rank:  0.7704612528095378
Raw Hits @1: 15.342519300302941
hits_filter Hits @1: 15.342519300302941
Raw Hits @10: 126.89338414932084
hits_filter Hits @10: 126.9422456757549
time: 39.7 s (started: 2022-12-19 17:03:10 +00:00)


In [46]:
encoded_dataset['test'].features

{'labels': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

time: 3.48 ms (started: 2022-12-19 13:44:44 +00:00)


In [70]:
logits

SequenceClassifierOutput(loss=None, logits=tensor([[-1.5013, -0.0273, -2.1696,  ..., -2.1861, -2.8690, -2.9332],
        [-1.5013, -0.0273, -2.1696,  ..., -2.1861, -2.8690, -2.9332],
        [-1.5013, -0.0273, -2.1696,  ..., -2.1861, -2.8690, -2.9332],
        ...,
        [-1.5013, -0.0273, -2.1696,  ..., -2.1861, -2.8690, -2.9332],
        [-1.5013, -0.0273, -2.1696,  ..., -2.1861, -2.8690, -2.9332],
        [-1.5013, -0.0273, -2.1696,  ..., -2.1861, -2.8690, -2.9332]],
       device='cuda:0'), hidden_states=None, attentions=None)

time: 32.4 ms (started: 2022-12-19 14:41:02 +00:00)


In [76]:
from transformers.data.data_collator import default_data_collator

test_dataloader = DataLoader(encoded_dataset["test"], collate_fn=default_data_collator, batch_size=128)
model.eval()  
for step, batch in enumerate(test_dataloader):
            outputs = model(**batch)
            preds = outputs.logits.detach().cpu().numpy()

time: 5min 49s (started: 2022-12-19 14:57:01 +00:00)


In [77]:
all_label_ids=[i for i in label_to_id.values()] #Different classes of labels for each relation 
all_label_ids=np.array(all_label_ids)
all_label_ids

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

time: 3.45 ms (started: 2022-12-19 15:02:50 +00:00)


In [78]:
batch_labels=batch["labels"].numpy()
batch_labels[0]

140

time: 2.2 ms (started: 2022-12-19 15:02:50 +00:00)


In [79]:
preds.shape

(114, 237)

time: 2.53 ms (started: 2022-12-19 15:02:50 +00:00)


In [80]:
preds

array([[-2.548286  , -2.4789112 , -3.988231  , ..., -0.68338597,
        -3.979366  , -3.6110728 ],
       [-2.0510328 , -1.3225387 , -3.2573605 , ..., -2.6216576 ,
        -2.3799894 , -2.529954  ],
       [-2.7583618 , -2.9967847 , -2.4148808 , ..., -2.7792912 ,
        -0.96308887, -4.7458057 ],
       ...,
       [-4.319901  , -0.7664732 , -4.5422163 , ..., -0.76863027,
        -2.6567495 , -1.9934225 ],
       [-2.6948097 , -3.037228  , -3.4647367 , ..., -5.6502075 ,
        -1.989789  , -4.080779  ],
       [-1.1739255 , -0.49074587, -2.458424  , ..., -1.2082027 ,
        -4.01051   , -5.389812  ]], dtype=float32)

time: 3.19 ms (started: 2022-12-19 15:02:50 +00:00)


In [ ]:
rel_values

tensor([-2.5483, -2.4789, -3.9882, -3.7459, -1.1194, -0.0850, -3.0508, -3.4064,
        -4.1653, -2.2771, -3.9549, -5.4905, -4.6560, -4.2216, -2.7382, -2.4039,
        -1.5762,  0.1525, -1.6004, -0.7449, -2.2484, -3.6342, -2.8314, -2.2867,
        -2.8729, -3.5022, -2.5718, -2.5405, -3.7058, -4.0884, -4.0893, -3.6867,
        -4.2607, -5.5486, -2.1355, -0.6892, -5.3034, -3.2622, -4.2946, -3.1429,
        -2.2996, -3.1100, -4.5746, -3.3171, -4.1091, -4.2904, -1.3892, -2.6125,
        -4.1590, -3.8463, -2.2607, -4.4302, -3.6187, -1.9932, -2.5312, -2.8056,
        -4.7516, -2.0021, -4.7896, -5.4156, -2.7639, -2.6574, -3.1777, -4.0106,
        -2.6310, -2.7283, -4.2796, -3.0285, -2.7645, -2.6639, -2.0087, -2.8244,
        -2.5340, -3.2299, -2.6286, -1.3975, -1.6000, -3.1568, -2.4731, -1.1791,
        -1.9272, -1.9017, -5.1346, -0.9456, -3.6841, -3.8177, -3.3539, -3.7676,
         4.6756,  0.1332, -4.4187, -5.9626, -4.4824, -5.6066, -1.1124, -3.8626,
         0.2383, -1.9442, -2.9671, -0.19

In [ ]:
argsort1[:]

array([140,  88, 172, 112, 225, 198, 190,  96, 197,  17,  89,   5, 157,
       203,  99, 167, 210, 234,  35,  19, 222, 201,  83,  94,   4, 137,
       174, 139,  79, 186, 104, 151,  46,  75, 224, 205, 133, 200, 211,
       127, 107, 231,  16,  76,  18, 192, 132, 187,  81, 131,  80,  97,
        53,  57,  70, 208, 147, 223, 229,  34,  20,  50, 153,   9,  23,
        40,  15, 185, 213,  78,   1, 184, 123,  54,  72,  27,   0,  26,
        47,  74,  64, 117, 193,  61,  69, 181, 168,  65,  14, 111,  60,
        68, 194,  55,  71,  22,  24, 160, 156, 118, 108, 124,  98, 191,
       233,  67, 212,   6, 150, 134,  41,  39,  77,  62, 113, 221, 199,
        73, 116, 162, 183,  37, 159, 125, 178,  43,  86, 166, 121,   7,
       232, 149, 218, 129, 180, 120, 144, 143,  25, 169, 170, 128, 146,
       130, 236, 175,  52, 135, 161,  21, 126, 110, 173,  84, 219,  31,
       202,  28,   3,  87,  85,  49,  95, 228, 105,  10, 122, 235,   2,
        63, 138, 148, 196, 136,  29,  30, 152, 207, 158,  44, 21

In [ ]:
all_triples_str_set

{'/m/02rdxsh\t/award/award_category/nominees./award/award_nomination/nominated_for\t/m/011ywj',
 '/m/0dm5l\t/award/award_nominee/award_nominations./award/award_nomination/award\t/m/02f705',
 '/m/016wvy\t/people/person/profession\t/m/0dz3r',
 '/m/0275n3y\t/award/award_ceremony/awards_presented./award/award_honor/award_winner\t/m/0gsg7',
 '/m/0gs9p\t/award/award_category/nominees./award/award_nomination/nominated_for\t/m/07xtqq',
 '/m/04sj3\t/olympics/olympic_participating_country/athletes./olympics/olympic_athlete_affiliation/olympics\t/m/06sks6',
 '/m/0c94fn\t/people/person/nationality\t/m/09c7w0',
 '/m/06xkst\t/tv/tv_program/genre\t/m/0jxy',
 '/m/02hct1\t/tv/tv_program/regular_cast./tv/regular_tv_appearance/actor\t/m/032xhg',
 '/m/01gjd0\t/military/military_conflict/combatants./military/military_combatant_group/combatants\t/m/0d05w3',
 '/m/03f0324\t/influence/influence_node/influenced_by\t/m/037jz',
 '/m/054k_8\t/people/person/profession\t/m/01d_h8',
 '/m/0f2w0\t/base/biblioness/bibs_

In [ ]:
batch_preds

array([-2.548286  , -2.4789112 , -3.988231  , -3.7458901 , -1.1194338 ,
       -0.08502534, -3.0508056 , -3.4064257 , -4.165345  , -2.2770615 ,
       -3.9548848 , -5.490492  , -4.6560087 , -4.22162   , -2.738243  ,
       -2.4038813 , -1.5761675 ,  0.1524665 , -1.6003966 , -0.7448543 ,
       -2.2484102 , -3.6341515 , -2.8314462 , -2.2866993 , -2.872923  ,
       -3.5022483 , -2.5718079 , -2.5404902 , -3.7057831 , -4.08842   ,
       -4.0892763 , -3.6866982 , -4.260681  , -5.548586  , -2.1355486 ,
       -0.6892233 , -5.303381  , -3.2622132 , -4.2946444 , -3.142868  ,
       -2.299646  , -3.1099954 , -4.574568  , -3.3171272 , -4.1091204 ,
       -4.2904267 , -1.3892443 , -2.612477  , -4.1590223 , -3.8463418 ,
       -2.2607315 , -4.4302163 , -3.6187372 , -1.993236  , -2.531236  ,
       -2.8055806 , -4.7516103 , -2.0020509 , -4.7895637 , -5.415636  ,
       -2.7638824 , -2.6573675 , -3.1777353 , -4.010555  , -2.631044  ,
       -2.7283435 , -4.279641  , -3.028531  , -2.7644782 , -2.66

In [105]:
model

DebertaForSequenceClassification(
  (deberta): DebertaModel(
    (embeddings): DebertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=0)
      (LayerNorm): DebertaLayerNorm()
      (dropout): StableDropout()
    )
    (encoder): DebertaEncoder(
      (layer): ModuleList(
        (0): DebertaLayer(
          (attention): DebertaAttention(
            (self): DisentangledSelfAttention(
              (in_proj): Linear(in_features=768, out_features=2304, bias=False)
              (pos_dropout): StableDropout()
              (pos_proj): Linear(in_features=768, out_features=768, bias=False)
              (pos_q_proj): Linear(in_features=768, out_features=768, bias=True)
              (dropout): StableDropout()
            )
            (output): DebertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): DebertaLayerNorm()
              (dropout): StableDropout()
            )
          )
          (intermed

time: 4.74 ms (started: 2022-12-19 16:35:43 +00:00)


# **XLNet Implementation**

In [ ]:
tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")

In [ ]:
encoded_dataset = dataset.map(lambda x: tokenizer(x['triple_texts'], truncation=True, padding='max_length', max_length=50,return_tensors='pt'), remove_columns=['guid', 'triple_texts'],batched=True)
encoded_dataset

  0%|          | 0/50 [00:00<?, ?ba/s]

  0%|          | 0/18 [00:00<?, ?ba/s]

  0%|          | 0/21 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
    eval: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 17535
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 20466
    })
})

In [ ]:
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=num_labels).to(device)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.weight', 'sequence_summary.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [ ]:


args = TrainingArguments(
    output_dir='./output_FB15K_xlnet_relationprediction_final/',          # output directory
    num_train_epochs=6,              # total # of training epochs
    per_device_train_batch_size=64,  # batch size per device during training
    per_device_eval_batch_size=128,   # batch size for evaluation
    warmup_steps=0.1,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs', 
    report_to = 'wandb',                     # enable logging to W&B
    overwrite_output_dir = True,
    learning_rate = 5e-5,  
    save_strategy="no",
     # we can customize learning rate
    evaluation_strategy = 'epoch',  
    logging_strategy = 'epoch',             
    load_best_model_at_end = False,
    metric_for_best_model = 'accuracy',
    greater_is_better = True,
    run_name = 'relation_prediction_xlnet_3'            # name of the W&B run
)

In [ ]:


'''metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    metric = evaluate.load("accuracy")
    return metric.compute(predictions=predictions, references=labels)'''



def compute_metrics(eval_pred):
  metric1 = evaluate.load("precision")
  metric2 = evaluate.load("recall")
  metric3 = evaluate.load("f1")
  metric4 = evaluate.load("accuracy")

  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  precision = metric1.compute(predictions=predictions, references=labels,average="weighted")["precision"]
  recall = metric2.compute(predictions=predictions, references=labels, average="weighted")["recall"]
  f1 = metric3.compute(predictions=predictions, references=labels, average="weighted")["f1"]
  accuracy = metric4.compute(predictions=predictions, references=labels)["accuracy"]

  return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}

In [ ]:
num_labels

237

In [ ]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, batch,return_outputs=False):
        num_labels=237
        #batch = tuple(t.to(device) for t in batch)
        # forward pass
        outputs = model(**batch)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, num_labels), batch['labels'].view(-1))
        return (loss, outputs) if return_outputs else loss

In [ ]:
from transformers.data.data_collator import default_data_collator
trainer = CustomTrainer(model = model,                  # model to be trained
    args = args,                    # training args
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["eval"],
    tokenizer=tokenizer,            # for padding batched data
    data_collator=default_data_collator,
    compute_metrics=compute_metrics)

In [ ]:
trainer.train()
trainer.save_model(f'./output_FB15K_xlnet_relationprediction/')

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 50000
  Num Epochs = 6
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 4692
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


wandb: Currently logged in as: anvesha. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,1.562100,0.500153,0.847159,0.873168,0.849919,0.873168
2,0.689800,0.369032,0.886796,0.894554,0.879803,0.894554
3,0.566000,0.299444,0.905435,0.912347,0.902261,0.912347
4,0.488200,0.280385,0.908860,0.917080,0.906746,0.917080
5,0.434300,0.273123,0.918620,0.919247,0.913573,0.919247
6,0.390600,0.266094,0.925317,0.922498,0.919878,0.922498


***** Running Evaluation *****
  Num examples = 17535
  Batch size = 128


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Evaluation *****
  Num examples = 17535
  Batch size = 128
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sk

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 17535
  Batch size = 128


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.27303388714790344,
 'eval_precision': 0.9181936329394337,
 'eval_recall': 0.918562874251497,
 'eval_f1': 0.9148248492140051,
 'eval_accuracy': 0.918562874251497,
 'eval_runtime': 19.2119,
 'eval_samples_per_second': 912.717,
 'eval_steps_per_second': 7.131,
 'epoch': 6.0}

In [ ]:
trainer.predict(encoded_dataset['test'])

***** Running Prediction *****
  Num examples = 20466
  Batch size = 128


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


PredictionOutput(predictions=array([[ 1.7239536 ,  0.4154079 , -0.09830011, ...,  0.92779434,
        -0.1064458 , -0.65740997],
       [ 0.01409622,  0.06440496,  0.22543935, ...,  2.9183156 ,
        -0.38511795, -0.9560745 ],
       [ 0.25444043,  1.9697292 , -1.6386719 , ...,  0.19360936,
        -3.1341627 , -1.9223729 ],
       ...,
       [-0.00968348, -1.0166323 , -1.0036882 , ...,  1.6740211 ,
        -0.23282012,  0.4058258 ],
       [-0.55751735, -0.22552545,  0.02874594, ..., -0.7485856 ,
         0.3570106 , -0.52179456],
       [ 1.0420161 , -1.0368689 , -2.6837547 , ...,  0.7415134 ,
        -2.241392  , -0.14259273]], dtype=float32), label_ids=array([ 52, 193, 190, ...,  49, 125, 198]), metrics={'test_loss': 0.26383471488952637, 'test_precision': 0.9195210221141406, 'test_recall': 0.9192807583308903, 'test_f1': 0.91599666636122, 'test_accuracy': 0.9192807583308903, 'test_runtime': 22.1909, 'test_samples_per_second': 922.269, 'test_steps_per_second': 7.21})

In [ ]:
tokenizer = XLNetTokenizer.from_pretrained("./output_FB15K_xlnet_relationprediction/")
model = XLNetForSequenceClassification.from_pretrained('./output_FB15K_xlnet_relationprediction/', num_labels=num_labels)

Didn't find file ./output_FB15K_xlnet_relationprediction/added_tokens.json. We won't load it.
loading file ./output_FB15K_xlnet_relationprediction/spiece.model
loading file None
loading file ./output_FB15K_xlnet_relationprediction/special_tokens_map.json
loading file ./output_FB15K_xlnet_relationprediction/tokenizer_config.json
loading configuration file ./output_FB15K_xlnet_relationprediction/config.json
Model config XLNetConfig {
  "_name_or_path": "xlnet-base-cased",
  "architectures": [
    "XLNetForSequenceClassification"
  ],
  "attn_type": "bi",
  "bi_data": false,
  "bos_token_id": 1,
  "clamp_len": -1,
  "d_head": 64,
  "d_inner": 3072,
  "d_model": 768,
  "dropout": 0.1,
  "end_n_top": 5,
  "eos_token_id": 2,
  "ff_activation": "gelu",
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "1

In [ ]:
encoded_dataset = dataset.map(lambda x: tokenizer(x['triple_texts'], truncation=True, padding='max_length', max_length=30,return_tensors='pt'), remove_columns=['guid', 'triple_texts'],batched=True)
encoded_dataset

  0%|          | 0/50 [00:00<?, ?ba/s]

  0%|          | 0/18 [00:00<?, ?ba/s]

  0%|          | 0/21 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
    eval: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 17535
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 20466
    })
})

In [ ]:
#Relation Prediction using test data 
from transformers.data.data_collator import default_data_collator

test_dataloader = DataLoader(encoded_dataset["test"], collate_fn=default_data_collator, batch_size=128)



In [ ]:
#test_dataloader = DataLoader(encoded_dataset["test"], collate_fn=data_collator, batch_size=512)

In [ ]:

model.eval()  
for step, batch in enumerate(test_dataloader):
            outputs = model(**batch)
            preds = outputs.logits.detach().cpu().numpy()

In [ ]:
all_label_ids=[i for i in label_to_id.values()] #Different classes of labels for each relation 
all_label_ids=np.array(all_label_ids)
all_label_ids

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [ ]:
batch_labels=batch["labels"].numpy()
batch_labels[0]

140

In [ ]:
preds.shape

(114, 237)

In [ ]:
# The entire Code is taken and adapted from the following source : "https://github.com/yao8839836/kg-bert"
train_triples = processor.get_train_triples(data_dir)
dev_triples = processor.get_dev_triples(data_dir)
test_triples = processor.get_test_triples(data_dir)
all_triples = train_triples + dev_triples + test_triples

all_triples_str_set = set()
for triple in all_triples:
            triple_str = '\t'.join(triple)
            all_triples_str_set.add(triple_str)
batch_preds = preds[0]
ranks = []
filter_ranks = []
hits = []
hits_filter = []
for i in range(10):
    hits.append([])
    hits_filter.append([])

for i, pred in enumerate(batch_preds):
  rel_values = torch.tensor(batch_preds)
  _, argsort1 = torch.sort(rel_values, descending=True)
  argsort1 = argsort1.cpu().numpy()
            
  rank = np.where(argsort1 == all_label_ids[i])[0][0]
            #print(argsort1, all_label_ids[i], rank)
  ranks.append(rank + 1)
  test_triple = test_triples[i]
  filter_rank = rank
  for tmp_label_id in argsort1[:rank]:
    tmp_label = label_list[tmp_label_id]
    tmp_triple = [test_triple[0], tmp_label, test_triple[2]]
                #print(tmp_triple)
    tmp_triple_str = '\t'.join(tmp_triple)
    if tmp_triple_str in all_triples_str_set:
        filter_rank -= 1
  filter_ranks.append(filter_rank + 1)

  for hits_level in range(10):
                if rank <= hits_level:
                    hits[hits_level].append(1.0)
                else:
                    hits[hits_level].append(0.0)

                if filter_rank <= hits_level:
                    hits_filter[hits_level].append(1.0)
                else:
                    hits_filter[hits_level].append(0.0)   

print("Raw mean rank: ", np.mean(ranks))
print("Filtered mean rank: ", np.mean(filter_ranks))
for i in [0,2,9]:
            print('Raw Hits @{0}: {1}'.format(i+1, np.mean(hits[i])))
            print('hits_filter Hits @{0}: {1}'.format(i+1, np.mean(hits_filter[i])))






    
   
     

Raw mean rank:  119.0
Filtered mean rank:  118.27004219409282
Raw Hits @1: 0.004219409282700422
hits_filter Hits @1: 0.004219409282700422
Raw Hits @3: 0.012658227848101266
hits_filter Hits @3: 0.012658227848101266
Raw Hits @10: 0.04219409282700422
hits_filter Hits @10: 0.04219409282700422


In [ ]:
rel_values

tensor([ 1.7760e-01, -7.1015e-01, -2.2710e+00,  4.7485e-01,  1.5675e+00,
         1.1852e+00, -3.5952e-02, -6.7979e-01, -6.6972e-01,  1.2615e+00,
        -8.2909e-02, -1.6218e-01, -1.9918e+00, -1.5609e+00, -5.4937e-01,
        -1.0006e+00, -1.1343e+00,  8.9786e-02, -4.6214e-01, -1.8950e-01,
         1.0615e+00, -1.0775e+00, -7.6149e-01,  1.2648e-01, -4.3211e-01,
        -1.1459e+00,  2.8545e-02, -1.2783e+00, -3.1359e-01, -1.9098e-02,
        -4.6900e-01,  3.9597e-01, -1.0411e+00, -1.0787e+00, -2.3052e-02,
         1.8907e-01, -5.9577e-01, -9.9691e-01, -1.0024e+00, -1.5325e+00,
         2.3147e-01, -2.0393e-01, -8.7669e-01, -1.2161e-01, -1.3498e+00,
        -4.4394e-01,  6.9143e-01, -1.5133e+00, -1.8846e-01, -1.7618e+00,
         2.9297e-01, -2.0756e-01,  9.8703e-01, -1.1207e+00,  2.6866e-01,
         2.2672e-01, -7.8016e-01,  2.7894e-01, -2.0967e+00, -2.9348e-01,
        -3.1597e-01,  9.1694e-01, -1.4688e-01, -7.9152e-01, -1.2000e+00,
        -5.9190e-01, -6.0279e-01, -1.0998e+00, -4.9

In [ ]:
argsort1[:]

array([140,  88, 104, 197, 198, 172, 203, 225,  99,   4, 167,  77, 151,
       157,   9, 231,   5, 139,  94, 132,  20,  96,  52, 201,  61, 147,
       191, 150, 209,  46, 135, 190, 219,  81, 221,  89,   3, 131, 200,
        31, 205, 234,  79,  50,  76,  57,  54, 223, 170,  40,  55,  35,
         0,  70, 137, 213,  23, 224, 111, 196, 181,  17, 226, 207, 124,
       176,  26, 187, 168,  29,  34, 204, 192,   6, 149, 184,  10,  43,
       174,  62, 107, 100,  11, 211,  48,  19,  73,  41,  51, 129, 183,
        75,  71, 163, 113, 112,  59, 178, 195,  28,  60, 146, 188, 125,
       158, 216,  24, 154, 130, 218,  45,  74,  18,  72,  30, 228,  68,
        69, 138,  83,  84,  14, 126, 109, 193,  65,  36,  66, 217, 199,
       116, 159, 118, 160,   8,   7, 108,  80,  95,   1,  22,  86, 153,
       103,  56, 175, 143, 186,  63, 233, 119,  82, 171, 134,  42, 222,
        85, 152, 101,  97, 220, 166, 185, 194,  37,  15,  38, 127, 148,
        32,  78,  21,  33,  67, 236, 214,  53, 206, 122,  16,  2

In [ ]:
all_triples_str_set

{'/m/024pcx\t/location/country/capital\t/m/0bwtj',
 '/m/03q2t9\t/award/award_nominee/award_nominations./award/award_nomination/award\t/m/01cw51',
 '/m/0bqvs2\t/award/award_nominee/award_nominations./award/award_nomination/award_nominee\t/m/01wqmm8',
 '/m/06ltr\t/film/actor/film./film/performance/film\t/m/016ywb',
 '/m/02rb84n\t/film/film/produced_by\t/m/06pj8',
 '/m/0gqyl\t/award/award_category/winners./award/award_honor/ceremony\t/m/0bzm81',
 '/m/015p37\t/people/person/profession\t/m/01d_h8',
 '/m/0l6m5\t/olympics/olympic_games/sports\t/m/0dwxr',
 '/m/0nm87\t/location/hud_foreclosure_area/estimated_number_of_mortgages./measurement_unit/dated_integer/source\t/m/0jbk9',
 '/m/039g82\t/award/award_nominee/award_nominations./award/award_nomination/award\t/m/09qv3c',
 '/m/09q17\t/media_common/netflix_genre/titles\t/m/01shy7',
 '/m/06sks6\t/olympics/olympic_games/participating_countries\t/m/06qd3',
 '/m/05q9g1\t/people/person/profession\t/m/025352',
 '/m/0bwgc_\t/people/person/gender\t/m/02z

In [ ]:
batch_preds

array([ 1.77596003e-01, -7.10152030e-01, -2.27098799e+00,  4.74852085e-01,
        1.56753135e+00,  1.18521476e+00, -3.59517932e-02, -6.79785430e-01,
       -6.69721484e-01,  1.26154768e+00, -8.29086602e-02, -1.62177712e-01,
       -1.99180818e+00, -1.56091046e+00, -5.49368382e-01, -1.00060630e+00,
       -1.13434052e+00,  8.97863507e-02, -4.62143958e-01, -1.89502433e-01,
        1.06151259e+00, -1.07754660e+00, -7.61485517e-01,  1.26476422e-01,
       -4.32114303e-01, -1.14589918e+00,  2.85446644e-02, -1.27831101e+00,
       -3.13593298e-01, -1.90981552e-02, -4.69004631e-01,  3.95970732e-01,
       -1.04113412e+00, -1.07867527e+00, -2.30516791e-02,  1.89074576e-01,
       -5.95765948e-01, -9.96905565e-01, -1.00239456e+00, -1.53249717e+00,
        2.31470391e-01, -2.03933775e-01, -8.76692653e-01, -1.21610135e-01,
       -1.34980452e+00, -4.43940490e-01,  6.91433728e-01, -1.51326275e+00,
       -1.88457772e-01, -1.76180398e+00,  2.92967498e-01, -2.07560122e-01,
        9.87029672e-01, -

In [ ]:
wandb.finish()

AttributeError: ignored